# Build and log the trip bot

A bot is here defined as an app which uses LLM. It can be an agent, 
if it takes autonomous decisions.

Logging enables us to track bot evaluations, and deploy the bot.

Run this notebook on the serverless cluster.

Select Serverless environment v3, from the settings button on the right side menu.

![Serverless env](https://raw.githubusercontent.com/brickops/databricks-botops-course/refs/heads/main/images/serverless_env_wide.png).

### Disclaimer: The current bot is a python tool bot, not a trip bot.

TODO: Hopefully it will be replaced before we run the workshop.
It should be a bot which queries trip data.

## Install dependencies

#### Common setup

You can ignore the message `Core Python package version(s) changed`

In [0]:
%run ../../../libs/botops/buildsetup_serverless

## Run bot

Run the bot in the context of this notebook, to see if it works

#### Set the bot name

Set bot name to the name of the current folder

In [0]:
BOT_NAME = folder()

## Study the bot.py file to see the structure of the agent itself

Apart from containing the bot-code,  the bot.py file is loaded by MLFlow to determine dependencies for bot.

## Try running the agent

In [0]:
from bot import BOT

BOT.predict({"messages": [{"role": "user", "content": "Which borrow has most taxi trips?"}]})


## Consume the predict_stream from the bot

It is used to stream partial response messages to give a better user experience in the chat bot.

In [0]:
for chunk in BOT.predict_stream(
    {"messages": [{"role": "user", "content": "Which borrow has most taxi trips?"}]}
):
    print(chunk, "-----------\n")

## Log the bot as an MLflow model

Log the bot as code from the `bot.py` file. See [MLflow - Models from Code](https://mlflow.org/docs/latest/models.html#models-from-code).

This enables us to track the version of the bot, and evaluations as experiments, as well as deploy it with unity catalog.

In [0]:
# Determine Databricks resources to specify for automatic auth passthrough at deployment time
from bot import LLM_ENDPOINT_NAME
import mlflow
from mlflow.models.resources import DatabricksFunction, DatabricksServingEndpoint

# Pass the llm endpoint as a resource to enable automatic authentication
resources = [DatabricksServingEndpoint(endpoint_name=LLM_ENDPOINT_NAME)]

with mlflow.start_run():  # A run enable mlflow registration, logging and tracking
    logged_agent_info = mlflow.pyfunc.log_model(
        artifact_path=BOT_NAME,
        python_model="bot.py",
        code_paths=["./brickbots"],
        pip_requirements=[
            "mlflow==3.4.0",
            "backoff==2.2.1",
            "databricks-openai==0.6.1",
        ],
        resources=resources,
    )

## Show the details of the logging

In [0]:
logged_agent_info.run_id

In [0]:
logged_agent_info.model_uri

In [0]:
logged_agent_info.artifact_path

In [0]:
logged_agent_info.metadata

Now let us evaluate our model before we deploy it, in the next notebook.

In [0]:
%sql
SHOW GRANTS ON CATALOG system;